## Kakao arena 2nd Competition
# "브런치 사용자를 위한 글 추천 대회"
### brunch 데이터를 활용해 사용자의 취향에 맞는 글을 예측하는 대회
* 공식 홈페이지: https://arena.kakao.com/c/2
* 베이스 코드: https://github.com/kakao-arena/brunch-article-recommendation

### BrunchRec 
* designed by **datartist**
* 깃헙 주소: https://github.com/jihoo-kim/BrunchRec  

## 1. 라이브러리 및 원본데이터

In [2]:
## 라이브러리
import numpy as np
import pandas as pd
import os
import time
import glob
import pickle
import datetime
import warnings
from itertools import chain
from collections import Counter

directory = './res/'
warnings.filterwarnings(action='ignore')

In [3]:
## 원본데이터

startTime = time.time()

# users  // DataFrame (310758, 3)
users = pd.read_json(directory + '/users.json', lines=True)

# magazine  // DataFrame (27967, 2)
magazine = pd.read_json(directory + 'magazine.json', lines=True)

# metadata  // DataFrame (643104, 9)
metadata = pd.read_json(directory + 'metadata.json', lines=True)

# dev.users  // List (3000)
f = open('./res/predict/dev.users')
dev_users = f.read().splitlines()
f.close()

# test.users  // List (5000)
f = open('./res/predict/test.users')
test_users = f.read().splitlines()
f.close()

# read  // DataFrame (3507097, 5)
read_file_lst = glob.glob('./res/read/*')
exclude_file_lst = ['read.tar']
read_df_lst = []
for f in read_file_lst:
    file_name = os.path.basename(f)
    if file_name in exclude_file_lst:
        print(file_name)
    else:
        df_temp = pd.read_csv(f, header=None, names=['raw'])
        df_temp['dt'] = file_name[:8]
        df_temp['hr'] = file_name[8:10]
        df_temp['user_id'] = df_temp['raw'].str.split(' ').str[0]
        df_temp['article_id'] = df_temp['raw'].str.split(' ').str[1:].str.join(' ').str.strip()
        read_df_lst.append(df_temp)
read = pd.concat(read_df_lst)

# read_raw  // DataFrame (22110706, 4)
def chainer(s):
    return list(chain.from_iterable(s.str.split(' ')))

read_cnt_by_user = read['article_id'].str.split(' ').map(len)

read_raw = pd.DataFrame({'dt': np.repeat(read['dt'], read_cnt_by_user),
                         'hr': np.repeat(read['hr'], read_cnt_by_user),
                         'user_id': np.repeat(read['user_id'], read_cnt_by_user),
                         'article_id': chainer(read['article_id'])})

endTime = time.time() - startTime
print(int(endTime), 'seconds', '=', int(endTime/60), 'minutes')

67 seconds = 1 minutes


## 2. 전처리 관련 함수

In [4]:
def get_unix_time(reg_ts):
    string_time = str(reg_ts)[:4]+'-'+str(reg_ts)[4:6]+'-'+str(reg_ts)[6:8]+' 00:00:00'
    unix_time = time.mktime(datetime.datetime.strptime(string_time, '%Y-%m-%d %H:%M:%S').timetuple())*1000
    
    return unix_time

In [23]:
## read_raw 'dt' 전처리 (날짜 컬럼의 자료형 변환 string -> int)
def read_raw_preprocessing(read_raw_df):
    
    if type(read_raw_df['dt'].values[0]) == type('string'):
        dt = read_raw_df['dt'].tolist()
        read_raw_df['dt'] = [int (i) for i in dt]
        print('read_raw_df preprocessing completed!')

    else:
        print('already preprocessed!')
        
    return read_raw_df

In [24]:
## metadata 'view' 전처리 (일정 기간 동안의 조회수 recent_view와 전체 기간 동안의 조회수 view 저장)
def metadata_preprocessing(metadata_df, read_raw_df, from_dt, to_dt):
    
    if 'recent_view' in metadata_df.keys():
        print('already preprocessed!')
        
    else:
        view = read_raw.groupby('article_id').count()['user_id']
        view_df = pd.DataFrame({'id':view.index, 'view':view.values})
        metadata_df = pd.merge(metadata_df, view_df, how='left', on='id')
        metadata_df['view'] = metadata_df['view'].fillna(0)
        
        partial_read = read_raw_df[(read_raw_df['dt'] >= from_dt) & (read_raw_df['dt'] <= to_dt)]
        recent_view = partial_read.groupby('article_id').count()['user_id']
        recent_view_df = pd.DataFrame({'id':recent_view.index, 'recent_view':recent_view.values})
        metadata_df = pd.merge(metadata_df, recent_view_df, how='left', on='id')
        metadata_df['recent_view'] = metadata_df['recent_view'].fillna(0)
        print('metadata_df preprocessing completed!')

    return metadata_df

In [7]:
## target_df 생성 (users에 없는 user_id 추가)
def target_df_generator(target_users_list, users_df):
    
    target_df = users_df[users_df['id'].isin(target_users_list)]
    
    for target_user in target_users_list:
        if (target_user in target_df['id'].tolist()) == False:
            new_df = pd.DataFrame({'following_list':[[]], 'id':[target_user], 'keyword_list':[[]]})
            target_df = target_df.append(new_df)
    
    return target_df

In [8]:
## target_df 'read' 전처리 (전체 기간 동안 target user가 본 article을 저장)
def target_read_article(target_df, read_raw_df, file):
    
    if os.path.isfile(file):        
        with open(file,"rb") as fr:
            target_read_article = pickle.load(fr)

        target_df['read'] = target_read_article
    
    else:
        target_read_article = []
        iteration = 0
        
        # target user가 전체 기간 동안 본 글의 article_id 저장 (중복 허용)
        for idx in target_df['id'].values.tolist():
            read_list = read_raw_df[read_raw_df['user_id']==idx]
            read_list = read_list[read_list['article_id'].str.startswith('@')]['article_id'].tolist()
            target_read_article.append(read_list)

            # 진행 상황 표시
            iteration += 1
            print(iteration, '/', str(len(target_df['id'].values.tolist())), 'completed')
            
        with open(file,"wb") as fw:
            pickle.dump(target_read_article, fw)
        
        target_df['read'] = target_read_article
            
    return target_df

In [9]:
## target_df 'recent' 전처리 (일정 기간 동안 target user가 본 article을 저장)
def target_recent_article(target_df, read_raw_df, from_dt, to_dt, file):
    
    if os.path.isfile(file):        
        with open(file,"rb") as fr:
            target_recent_article = pickle.load(fr)

        target_df['recent'] = target_recent_article
    
    else:
        target_recent_article = []
        iteration = 0
        
        # target user가 일정 기간 동안 본 글의 article_id 저장 (중복 허용)
        partial_read = read_raw_df[(read_raw_df['dt'] >= from_dt) & (read_raw_df['dt'] <= to_dt)]
        
        for idx in target_df['id'].values.tolist():    
            read_list = partial_read[partial_read['user_id']==idx]
            read_list = read_list[read_list['article_id'].str.startswith('@')]['article_id'].tolist()
            target_recent_article.append(read_list)

            # 진행 상황 표시
            iteration += 1
            print(iteration, '/', str(len(target_df['id'].values.tolist())), 'completed')
            
        with open(file,"wb") as fw:
            pickle.dump(target_recent_article, fw)
        
        target_df['recent'] = target_recent_article
            
    return target_df

In [10]:
# target_df 'following' 전처리 (target user가 본 following의 빈도수 저장)
def target_read_following(target_df, mode, file):
    
    if os.path.isfile(file):
        with open(file,"rb") as fr:
            target_read_following = pickle.load(fr)
        target_df[mode+'_following'] = target_read_following
    
    else:
        target_read_following = []
        iteration = 0

        for idx in target_df['id'].values.tolist():
            # following_frequency -> read_list 중에서 해당 작가(f_id)의 글의 빈도수
            following_frequency = {}
            
            # read_list- > target user의 read_list
            read_list = target_df[target_df['id']==idx][mode].values[0][:]
            
            if len(read_list) > 0:
                read_series = pd.Series(read_list)

                # f_list -> target user의 following_list (구독작가 리스트)
                f_list = target_df[target_df['id']==idx]['following_list'].values[0][:]  
                for i in range(len(f_list)):
                    f_list[i] = f_list[i] + '_'

                for f_id in f_list:
                    frequency = len(read_series[read_series.str.startswith(f_id)].tolist())
                    if frequency > 0:
                        following_frequency[f_id[:-1]]=frequency
                        
            target_read_following.append(following_frequency)

            # 진행 상황 표시
            iteration += 1
            print(iteration, '/', str(len(target_df['id'].values.tolist())), 'completed')
            
        with open(file,"wb") as fw:
            pickle.dump(target_read_following, fw)
        target_df[mode+'_following'] = target_read_following
    
    return target_df

In [11]:
# target_df 'magazine' 전처리 (target user가 본 magazine의 빈도수 저장)
def target_read_magazine(target_df, metadata_df, mode, file):
    
    if os.path.isfile(file):
        with open(file,"rb") as fr:
            target_read_magazine = pickle.load(fr)
        target_df[mode+'_magazine'] = target_read_magazine
    
    else:
        target_read_magazine = []
        iteration = 0

        for idx in target_df['id'].values.tolist():
            # target user가 읽은 글의 magazine_id 저장
            read_list = target_df[target_df['id']==idx][mode].values[0][:]
            magazine_list = metadata_df[metadata_df['id'].isin(read_list)]['magazine_id'].tolist()
            
            # magazine_id 빈도수 저장 (magazine이 아닌 0은 제외)
            magazine_frequency = Counter(magazine_list)
            del magazine_frequency[0]
            target_read_magazine.append(magazine_frequency)

            # 진행 상황 표시
            iteration += 1
            print(iteration, '/', str(len(target_df['id'].values.tolist())), 'completed')
            
        with open(file,"wb") as fw:
            pickle.dump(target_read_magazine, fw)
        target_df[mode+'_magazine'] = target_read_magazine
    
    return target_df

In [12]:
## target_df 'tag' 전처리 (target user가 본 article의 tag 빈도수 저장)
def target_read_tag(target_df, metadata_df, mode, file):
    
    if os.path.isfile(file):
        with open(file,"rb") as fr:
            target_read_tag = pickle.load(fr)
        target_df[mode+'_tag'] = target_read_tag
    
    else:
        target_read_tag = []
        iteration = 0

        for idx in target_df['id'].values.tolist():
            # read_list- > target user의 read_list
            read_list = target_df[target_df['id']==idx][mode].values[0][:]
            
            # 각 target user가 일정 기간 동안 읽은 글의 태그 합쳐서 저장 (중복 허용)
            keyword_list = metadata_df[metadata_df['id'].isin(read_list)]['keyword_list'].tolist()
            read_tag = sum(keyword_list, [])
            
            # 각 target user가 일정 기간 동안 읽은 글의 태그들의 빈도수 저장
            frequency = Counter(read_tag)
            target_read_tag.append(frequency)

            # 진행 상황 표시
            iteration += 1
            print(iteration, '/', str(len(target_df['id'].values.tolist())), 'completed')
        
        with open(file,"wb") as fw:
            pickle.dump(target_read_tag, fw)
        target_df[mode+'_tag'] = target_read_tag

    return target_df

In [13]:
## target_df 'interest' 전처리 (tag에서 빈도수가 높은 상위 top_N개 관심키워드 저장)
def target_read_interest(target_df, top_N, mode, file):
    
    if os.path.isfile(file):
        with open(file,"rb") as fr:
            target_read_interest = pickle.load(fr)
        target_df[mode+'_interest'] = target_read_interest
    
    else:
        target_read_interest = []
        iteration = 0      
        
        # read_tag에서 빈도수가 높은 상위 top_N개의 키워드 저장
        for idx in target_df['id'].values.tolist():
            interest = []
            
            rt = target_df[target_df['id']==idx][mode+'_tag'].values[0]
            sorted_rt = sorted(rt.items(), key=lambda x: x[1], reverse=True)
            
            for i in range(len(sorted_rt[:top_N])):
                interest.append(sorted_rt[:top_N][i][0])
                
            target_read_interest.append(interest)
            
            # 진행 상황 표시
            iteration += 1
            print(iteration, '/', str(len(target_df['id'].values.tolist())), 'completed')
        
        with open(file,"wb") as fw:
            pickle.dump(target_read_interest, fw)
        target_df[mode+'_interest'] = target_read_interest

    return target_df

In [14]:
## target_df 'behavior' 전처리 (target user의 글 소비 성향 저장)
def target_read_behavior(target_df, metadata_df, mode, file):
    
    if os.path.isfile(file):
        with open(file,"rb") as fr:
            ratio_list = pickle.load(fr)

    else:
        ratio_list = []

        if mode == 'recent':
            # 최근 기간 -> pop: view가 상위 1%인 글 / reg: 추천 기간 포함 1개월 동안 발행된 글
            pop_id = metadata_df[metadata_df['recent_view'] > metadata_df['recent_view'].quantile(0.99)]['id'].tolist()
            reg_id = metadata_df[metadata_df['reg_ts'] >= get_unix_time(20190215)]['id'].tolist()
        else:
            # 전체 기간 -> pop: view가 상위 20%인 글 / reg: 추천 기간 포함 6개월 동안 발행된 글
            pop_id = metadata_df[metadata_df['view'] > metadata_df['view'].quantile(0.80)]['id'].tolist()
            reg_id = metadata_df[metadata_df['reg_ts'] >= get_unix_time(20180915)]['id'].tolist()

        iteration = 0

        for idx in target_df['id'].values.tolist():
            read_list = target_df[target_df['id']==idx][mode].values[0][:]

            fr_dic = target_df[target_df['id']==idx][mode+'_following'].values[0]
            mr_dic = target_df[target_df['id']==idx][mode+'_magazine'].values[0]
            pop = pd.Series(read_list)[pd.Series(read_list).isin(pop_id)].tolist()
            reg = pd.Series(read_list)[pd.Series(read_list).isin(reg_id)].tolist()

            # f_ratio -> target user가 본 article 중에서 following의 비율
            # m_ratio -> target user가 본 article 중에서 magazine의 비율
            # p_ratio -> target user가 본 article 중에서 인기가 많은 글(view가 상위 1% or 20%)의 비율
            # r_ratio -> target user가 본 article 중에서 최근 발행된 글(reg_ts > 190215 or 180915)의 비율
            f_ratio, m_ratio, p_ratio, r_ratio = 0, 0, 0 ,0

            if len(read_list) >= 1:
                f_ratio = round(sum(fr_dic.values())/len(read_list), 2)
                m_ratio = round(sum(mr_dic.values())/len(read_list), 2)      
                p_ratio = round(len(pop)/len(read_list), 2)
                r_ratio = round(len(reg)/len(read_list), 2)

            ratio_list.append([f_ratio, m_ratio, p_ratio, r_ratio])

            # 진행 상황 표시
            iteration += 1
            print(iteration, '/', str(len(target_df['id'].values.tolist())), 'completed')
            
        with open(file,"wb") as fw:
            pickle.dump(ratio_list, fw)

    ratio_df = pd.DataFrame(ratio_list, columns=[mode+'_f_ratio', mode+'_m_ratio', mode+'_p_ratio', mode+'_r_ratio'])
    target_df.index = list(range(len(target_df)))
    target_df = pd.concat([target_df, ratio_df], axis=1)
    
    return target_df

In [15]:
# target user에 대한 전처리
def target_users_preprocessing(target_users_list, users_df, metadata_df, read_raw_df, from_dt, to_dt):
    
    target_df = target_df_generator(target_users_list, users)
    
    if target_users_list == dev_users:
        target = 'dev'
    elif target_users_list == test_users:
        target = 'test'

    # target user가 전체 기간 및 일정 기간 동안 읽은 글 저장
    target_df = target_read_article(target_df, read_raw_df, './pickle/'+target+'_read_article')
    target_df = target_recent_article(target_df, read_raw_df, from_dt, to_dt, './pickle/'+target+'_recent_article_'+str(from_dt)[2:])

    # target user가 읽은 글의 following 빈도수 저장
    target_df = target_read_following(target_df, 'read', './pickle/'+target+'_read_following')
    target_df = target_read_following(target_df, 'recent', './pickle/'+target+'_recent_following_'+str(from_dt)[2:])

    # target user가 읽은 글의 magazine 빈도수 저장
    target_df = target_read_magazine(target_df, metadata_df, 'read', './pickle/'+target+'_read_magazine')
    target_df = target_read_magazine(target_df, metadata_df, 'recent', './pickle/'+target+'_recent_magazine_'+str(from_dt)[2:])

    # target user가 읽은 글의 tag 빈도수 저장
    target_df = target_read_tag(target_df, metadata_df, 'read', './pickle/'+target+'_read_tag')
    target_df = target_read_tag(target_df, metadata_df, 'recent', './pickle/'+target+'_recent_tag_'+str(from_dt)[2:])

    # target user가 읽은 글의 tag 중 빈도수가 높은 top_N(=6)개의 interest 저장
    target_df = target_read_interest(target_df, 6, 'read', './pickle/'+target+'_read_interest')
    target_df = target_read_interest(target_df, 6, 'recent', './pickle/'+target+'_recent_interest_'+str(from_dt)[2:])

    # target user의 글 소비 경향 저장
    target_df = target_read_behavior(target_df, metadata_df, 'read', './pickle/'+target+'_read_behavior')
    target_df = target_read_behavior(target_df, metadata_df, 'recent', './pickle/'+target+'_recent_behavior')
    
    print('target_users_df preprocessing completed!')

    return target_df

## 3. 추천 관련 함수

In [47]:
# 일정 기간 동안 조회수가 높은 인기 글을 추천
def popularity_based_recommend(idx, target_df, metadata_df, read_raw_df, r_list, recommended, p_ratio):
    
    already = r_list + recommended
    
    # 이미 본 글이나 추천된 글을 제외하고 view 순으로 n_rec만큼 추천
    n_rec = int((100-len(recommended)) * p_ratio)
    p_article = metadata_df[metadata_df['id'].isin(already)==False].sort_values(['view'],ascending=[False])
    popularity_based_recommend_list = p_article[:n_rec]['id'].tolist()
    
    return popularity_based_recommend_list

In [48]:
# target user가 최근 또는 전체 기간 동안 읽은 글 중에서 구독작가 글의 비율을 고려하여 추천
def following_based_recommend(idx, target_df, metadata_df, r_list, recommended, mode, f_ratio, sort_option):
    
    already = r_list + recommended
    following_based_recommend_list = []
    
    # fr_dic -> {f_id,빈도수}
    if mode == 'recent':
        fr_dic = target_df[target_df['id']==idx]['recent_following'].values[0]
    if mode == 'read':
        fr_dic = target_df[target_df['id']==idx]['read_following'].values[0]
        
    # sorted_fr -> 빈도수 순으로 정렬
    sorted_fr = sorted(fr_dic.items(), key=lambda x: x[1], reverse=True)
    
    for i in range(len(sorted_fr)):
        if sorted_fr[i][1] > 0:
            # n_rec -> 추천할 구독작가의 글의 개수 (빈도수가 높을수록 많이 추천됨)
            n_rec = int( (100-len(recommended)) * f_ratio * (sorted_fr[i][1]/sum(fr_dic.values())) )
            fr_article = metadata_df[metadata_df['user_id']==sorted_fr[i][0]]
            # 이미 본 글이나 이미 추천된 글을 제외하고 sort_option 순으로 n_rec만큼 추천
            fr_candidate = fr_article[fr_article['id'].isin(already)==False].sort_values([sort_option],ascending=[False])[:n_rec]['id'].tolist()
            following_based_recommend_list = following_based_recommend_list + fr_candidate
    
    return following_based_recommend_list

In [49]:
# target user가 최근에 읽은 글 중에서 매거진 글의 비율을 고려하여 추천
def magazine_based_recommend(idx, target_df, metadata_df, r_list, recommended, mode, m_ratio, sort_option):
    
    already = r_list + recommended
    magazine_based_recommend_list = []
    
    # mr_dic -> {m_id : 빈도수}
    if mode == 'recent':
        mr_dic = target_df[target_df['id']==idx]['recent_magazine'].values[0]
    if mode == 'read':
        mr_dic = target_df[target_df['id']==idx]['read_magazine'].values[0]
    
    # sorted_mr -> 빈도수 순으로 정렬
    sorted_mr = sorted(mr_dic.items(), key=lambda x: x[1], reverse=True)

    for i in range(len(sorted_mr)):
        # n_rec -> 추천할 매거진 글의 개수 (빈도수가 높을수록 많이 추천됨)
        n_rec = int( (100-len(recommended)) * m_ratio * (sorted_mr[i][1]/sum(mr_dic.values())) )
        mr_article = metadata_df[metadata_df['magazine_id']==sorted_mr[i][0]]
        # 이미 본 글이나 이미 추천된 글을 제외하고 sort_option 순으로 n_rec만큼 추천
        mr_candidate = mr_article[mr_article['id'].isin(already)==False].sort_values([sort_option],ascending=[False])[:n_rec]['id'].tolist()
        magazine_based_recommend_list = magazine_based_recommend_list + mr_candidate
    
    return magazine_based_recommend_list

In [50]:
# target user가 최근에 읽은 글들에서 자주 나오는 태그를 고려하여 추천
def tag_based_recommend(idx, target_df, metadata_df, r_list, recommended, recommend_list, mode, common_num, t_ratio, sort_option):
    
    already = r_list + recommended + sum(recommend_list, [])

    # user_interest -> target user의 interest (0 ~ 6개)
    if mode == 'recent':
        user_interest = target_df[target_df['id']== idx]['recent_interest'].values[0][:]
    if mode == 'read':
        user_interest = target_df[target_df['id']== idx]['read_interest'].values[0][:]

    # interest_article_id -> target user의 interest와 common_num개 이상 겹치는 글의 article_id
    interest_article_id = []
    for i in range(len(metadata_df)):
        if len(set(metadata_df['keyword_list'].values[i]) & set(user_interest)) >= common_num:
            interest_article_id.append(metadata_df['id'].values[i])

    # 이미 본 글이나 이미 추천된 글을 제외하고 sort_option 순으로 n_rec만큼 추천
    n_rec = int((100-len(recommended)) * t_ratio)
    t_article = metadata_df[metadata_df['id'].isin(interest_article_id)]
    tag_based_recommend_list = t_article[t_article['id'].isin(already)==False].sort_values([sort_option],ascending=[False])[:n_rec]['id'].tolist()

    return tag_based_recommend_list

In [51]:
def behavior_based_recommend(idx, target_df, metadata_all, metadata_reg, metadata_pop, metadata_hot, read_raw_df, r_list, recommended, recommend_list, mode, f_ratio, m_ratio):    
    
    f1 = following_based_recommend(idx, target_df, metadata_hot, r_list, recommended, mode, 1, 'view')
    recommended = recommended + f1

    f2 = following_based_recommend(idx, target_df, metadata_reg, r_list, recommended, mode, 1, 'reg_ts')
    recommended = recommended + f2
    
    p1 = popularity_based_recommend(idx, target_df, metadata_pop, read_raw_df, r_list, recommended, 0.1)
    recommended = recommended + p1

    m1 = magazine_based_recommend(idx, target_df, metadata_hot, r_list, recommended, mode, 0.5, 'view')
    recommended = recommended + m1
    
    t1 = tag_based_recommend(idx, target_df, metadata_hot, r_list, recommended, recommend_list, mode, 3, 1, 'view')
    recommended = recommended + t1
    
    m2 = magazine_based_recommend(idx, target_df, metadata_all, r_list, recommended, mode, 1, 'reg_ts')
    recommended = recommended + m2

    t2 = tag_based_recommend(idx, target_df, metadata_reg, r_list, recommended, recommend_list, mode, 2, 1, 'reg_ts')
    recommended = recommended + t2

    print('fr:'+str(int(f_ratio*100)).rjust(3)+'%,', 'mr:'+str(int(m_ratio*100)).rjust(3)+'%', '\t', \
          'f1:'+str(len(f1)).rjust(3)+',', 'f2:'+str(len(f2)).rjust(3)+',', 'p1:'+str(len(p1)).rjust(3)+',', \
          'm1:'+str(len(m1)).rjust(3)+',', 'm2:'+str(len(m2)).rjust(3)+',', \
          't1:'+str(len(t1)).rjust(3)+',', 't2:'+str(len(t2)).rjust(3)+',', 'p2:'+str(100-len(recommended)).rjust(3))

    return recommended

In [52]:
def recommender(target_list, target_df, metadata_df, read_raw_df, output_file):
    
    # 추천 실행 시간 측정
    startTime = time.time()
    # 최종 추천 리스트
    recommend_list = []
    
    # metadata_all -> 추천 기간 이후에 발행된 글을 제외한 metadata
    metadata_all = metadata_df[metadata_df['reg_ts'] < get_unix_time(20190314)]
    # metadata_reg -> 추천 기간을 포함한 최근 6개월 동안 발행된 글의 metadata
    metadata_reg = metadata_df[(metadata_df['reg_ts'] >= get_unix_time(20180914)) & (metadata_df['reg_ts'] < get_unix_time(20190314))]
    # metadata_pop -> 일정 기간 동안 view가 상위 20%인 글의 metadata
    metadata_pop = metadata_df[metadata_df['view'] > metadata_df['view'].quantile(0.80)]
    # metadata_hot -> 추천 기간 동안 발행되었고, view가 상위 1%인 글의 metadata
    metadata_hot = metadata_df[(metadata_df['view'] > metadata_df['view'].quantile(0.99)) & ((metadata_df['reg_ts'] >= get_unix_time(20190222)))]
 
    
    # recent_min -> 최소 recent 건수
    n_recent = []
    for i in range(len(target_df)):
        n_recent.append(len(target_df['recent'].values[i]))
    recent_min = np.percentile(np.array(n_recent), 20)

    # read_min -> 최소 read 건수
    n_read = []
    for i in range(len(target_df)):
        n_read.append(len(target_df['read'].values[i]))
    read_min = np.percentile(np.array(n_read), 20)
    
    # 진행 상황
    iteration = 0
    
    for idx in target_list:
        recommended = []

        r_list = target_df[target_df['id']==idx]['read'].values[0][:]
        recent = target_df[target_df['id']==idx]['recent'].values[0][:]
        read_f_ratio = target_df[target_df['id']==idx]['read_f_ratio'].values[0]
        read_m_ratio = target_df[target_df['id']==idx]['read_m_ratio'].values[0]
        recent_f_ratio = target_df[target_df['id']==idx]['recent_f_ratio'].values[0]
        recent_m_ratio = target_df[target_df['id']==idx]['recent_m_ratio'].values[0]

        
        if len(recent) > recent_min:
            print('read:'+str(len(r_list))+',', 'recent:'+str(len(recent)), '\t', 'mode: recent')
            recommended = behavior_based_recommend(idx, target_df, metadata_all, metadata_reg, metadata_pop, metadata_hot, read_raw_df, r_list, recommended, recommend_list, 'recent', recent_f_ratio, recent_m_ratio)
                
        elif len(r_list) > read_min:
            print('read:'+str(len(r_list))+',', 'recent:'+str(len(recent)), '\t', 'mode: read')
            recommended = behavior_based_recommend(idx, target_df, metadata_all, metadata_reg, metadata_pop, metadata_hot, read_raw_df, r_list, recommended, recommend_list, 'read', read_f_ratio, read_m_ratio)
            
        else:
            print('read:'+str(len(r_list))+',', 'recent:'+str(len(recent)), '\t', 'mode: else')
            recommended = behavior_based_recommend(idx, target_df, metadata_all, metadata_reg, metadata_pop, metadata_hot, read_raw_df, r_list, recommended, recommend_list, 'read', read_f_ratio, read_m_ratio)

        
        # 100개 되지 않았다면 popularity_based 추천
        if len(recommended) < 100:
            p2 = popularity_based_recommend(idx, target_df, metadata_hot, read_raw_df, r_list, recommended, 1.0)
            recommended = recommended + p2
            
        # 추천 리스트 맨 앞에 user_id 추가
        recommended.insert(0, idx)                
        recommend_list.append(recommended)        

        # 진행 상황 표시
        iteration += 1
        print('\t\t\t', 'Total:'+str(len(recommended)-1), '\t\t\t\t\t', str(iteration).rjust(4), '/', str(len(target_list)), 'completed', '\n')


    # 추천 리스트를 파일로 저장
    f = open(output_file, 'w')
    for i in range(len(recommend_list)):
        for j in range(len(recommend_list[i])):
            f.write(recommend_list[i][j])
            if j == (len(recommend_list[i]) - 1):
                continue
            f.write(' ')
        f.write('\n')
    f.close()
    print('recommend.txt file saved..')
    print('completed!')
    
    endTime = time.time() - startTime
    print(int(endTime), 'seconds', '=', int(endTime/60), 'minutes')

    return recommend_list

## 4. 메인

### Step 1. 전처리

In [25]:
startTime = time.time()

read_raw = read_raw_preprocessing(read_raw)
metadata = metadata_preprocessing(metadata, read_raw, 20190215, 20190228)
dev = target_users_preprocessing(dev_users, users, metadata, read_raw, 20190215, 20190228)

endTime = time.time() - startTime
print(int(endTime), 'seconds', '=', int(endTime/60), 'minutes')

already preprocessed!
already preprocessed!
target_users_df preprocessing completed!
1 seconds = 0 minutes


### Step 2. 추천

In [ ]:
recommend = recommender(dev_users, dev, metadata, read_raw, './recommend.txt')

read:103, recent:28 	 mode: recent
fr: 14%, mr: 43% 	 f1:  1, f2:  2, p1:  9, m1:  1, m2: 75, t1:  0, t2: 12, p2:  0
			 Total:100 					    1 / 3000 completed 

read:44, recent:3 	 mode: read
fr: 73%, mr: 86% 	 f1:  2, f2: 87, p1:  1, m1:  0, m2:  2, t1:  0, t2:  8, p2:  0
			 Total:100 					    2 / 3000 completed 

read:23, recent:3 	 mode: else
fr: 61%, mr: 48% 	 f1:  1, f2:  8, p1:  9, m1:  0, m2: 33, t1:  0, t2: 13, p2: 36
			 Total:100 					    3 / 3000 completed 

read:59, recent:20 	 mode: recent
fr: 80%, mr:  5% 	 f1:  1, f2:  9, p1:  9, m1:  0, m2: 81, t1:  0, t2:  0, p2:  0
			 Total:100 					    4 / 3000 completed 

read:413, recent:36 	 mode: recent
fr: 78%, mr: 22% 	 f1:  0, f2: 57, p1:  4, m1:  0, m2: 28, t1:  0, t2: 11, p2:  0
			 Total:100 					    5 / 3000 completed 

read:5, recent:5 	 mode: else
fr:100%, mr: 40% 	 f1:  1, f2: 20, p1:  7, m1:  0, m2:  0, t1:  0, t2:  7, p2: 65
			 Total:100 					    6 / 3000 completed 

read:95, recent:14 	 mode: recent
fr: 86%, mr: 2

fr: 83%, mr: 25% 	 f1:  1, f2: 15, p1:  8, m1:  0, m2: 69, t1:  0, t2:  7, p2:  0
			 Total:100 					   53 / 3000 completed 

read:174, recent:174 	 mode: recent
fr: 89%, mr: 40% 	 f1:  0, f2: 99, p1:  0, m1:  0, m2:  0, t1:  0, t2:  1, p2:  0
			 Total:100 					   54 / 3000 completed 

read:106, recent:70 	 mode: recent
fr: 73%, mr: 46% 	 f1:  0, f2: 38, p1:  6, m1:  0, m2: 41, t1:  0, t2: 15, p2:  0
			 Total:100 					   55 / 3000 completed 

read:379, recent:11 	 mode: recent
fr: 27%, mr: 55% 	 f1:  0, f2: 49, p1:  5, m1:  0, m2: 43, t1:  0, t2:  3, p2:  0
			 Total:100 					   56 / 3000 completed 

read:8, recent:8 	 mode: recent
fr: 25%, mr: 75% 	 f1:  5, f2: 14, p1:  8, m1:  0, m2: 72, t1:  0, t2:  0, p2:  1
			 Total:100 					   57 / 3000 completed 

read:391, recent:5 	 mode: read
fr:  6%, mr: 55% 	 f1:  1, f2: 35, p1:  6, m1:  0, m2:  0, t1:  0, t2: 58, p2:  0
			 Total:100 					   58 / 3000 completed 

read:134, recent:13 	 mode: recent
fr: 62%, mr: 54% 	 f1:  2, f2: 21, p1: 

fr: 93%, mr: 34% 	 f1:  1, f2: 77, p1:  2, m1:  0, m2: 10, t1:  0, t2: 10, p2:  0
			 Total:100 					  105 / 3000 completed 

read:1474, recent:12 	 mode: recent
fr:100%, mr: 33% 	 f1:  0, f2: 58, p1:  4, m1:  0, m2: 16, t1:  0, t2: 22, p2:  0
			 Total:100 					  106 / 3000 completed 

read:6, recent:6 	 mode: else
fr:100%, mr: 67% 	 f1:  1, f2: 24, p1:  7, m1:  0, m2:  0, t1:  0, t2: 52, p2: 16
			 Total:100 					  107 / 3000 completed 

read:18, recent:2 	 mode: else
fr: 83%, mr: 83% 	 f1:  1, f2: 20, p1:  7, m1:  0, m2: 12, t1:  0, t2: 60, p2:  0
			 Total:100 					  108 / 3000 completed 

read:451, recent:89 	 mode: recent
fr: 89%, mr: 31% 	 f1:  1, f2: 39, p1:  6, m1:  0, m2: 13, t1:  0, t2: 41, p2:  0
			 Total:100 					  109 / 3000 completed 

read:585, recent:68 	 mode: recent
fr: 44%, mr: 54% 	 f1:  7, f2: 40, p1:  5, m1:  0, m2: 34, t1:  0, t2: 14, p2:  0
			 Total:100 					  110 / 3000 completed 

read:255, recent:70 	 mode: recent
fr:  0%, mr: 39% 	 f1:  0, f2:  0, p1: 10,

fr: 82%, mr: 31% 	 f1:  1, f2: 60, p1:  3, m1:  0, m2: 14, t1:  0, t2: 22, p2:  0
			 Total:100 					  157 / 3000 completed 

read:0, recent:0 	 mode: else
fr:  0%, mr:  0% 	 f1:  0, f2:  0, p1: 10, m1:  0, m2:  0, t1:  0, t2:  0, p2: 90
			 Total:100 					  158 / 3000 completed 

read:97, recent:32 	 mode: recent
fr: 25%, mr: 25% 	 f1:  1, f2: 59, p1:  4, m1:  0, m2: 33, t1:  0, t2:  3, p2:  0
			 Total:100 					  159 / 3000 completed 

read:415, recent:292 	 mode: recent
fr: 11%, mr: 47% 	 f1:  2, f2: 67, p1:  3, m1:  0, m2:  0, t1:  0, t2: 28, p2:  0
			 Total:100 					  160 / 3000 completed 

read:563, recent:50 	 mode: recent
fr: 10%, mr: 56% 	 f1:  1, f2:  9, p1:  9, m1:  5, m2: 56, t1:  0, t2: 20, p2:  0
			 Total:100 					  161 / 3000 completed 

read:0, recent:0 	 mode: else
fr:  0%, mr:  0% 	 f1:  0, f2:  0, p1: 10, m1:  0, m2:  0, t1:  0, t2:  0, p2: 90
			 Total:100 					  162 / 3000 completed 

read:2639, recent:256 	 mode: recent
fr: 44%, mr: 28% 	 f1:  3, f2: 82, p1:  1,

fr:  7%, mr: 36% 	 f1:  1, f2:  9, p1:  9, m1:  2, m2: 40, t1:  0, t2: 39, p2:  0
			 Total:100 					  209 / 3000 completed 

read:10, recent:8 	 mode: recent
fr: 62%, mr: 25% 	 f1:  0, f2:  3, p1:  9, m1:  0, m2: 71, t1:  0, t2: 17, p2:  0
			 Total:100 					  210 / 3000 completed 

read:2023, recent:319 	 mode: recent
fr: 62%, mr: 27% 	 f1:  1, f2: 47, p1:  5, m1:  0, m2: 15, t1:  0, t2: 32, p2:  0
			 Total:100 					  211 / 3000 completed 

read:531, recent:72 	 mode: recent
fr: 44%, mr: 60% 	 f1:  2, f2: 67, p1:  3, m1:  0, m2:  4, t1:  0, t2: 24, p2:  0
			 Total:100 					  212 / 3000 completed 

read:10, recent:9 	 mode: recent
fr:  0%, mr: 56% 	 f1:  0, f2:  0, p1: 10, m1:  0, m2: 48, t1:  0, t2: 29, p2: 13
			 Total:100 					  213 / 3000 completed 

read:838, recent:280 	 mode: recent
fr: 55%, mr: 27% 	 f1:  1, f2: 79, p1:  2, m1:  0, m2:  4, t1:  0, t2: 14, p2:  0
			 Total:100 					  214 / 3000 completed 

read:466, recent:55 	 mode: recent
fr: 35%, mr: 56% 	 f1:  1, f2: 31, 

...

fr:  0%, mr: 64% 	 f1:  0, f2:  0, p1: 10, m1:  0, m2: 54, t1:  0, t2: 36, p2:  0
			 Total:100 					  572 / 3000 completed 

read:7, recent:7 	 mode: recent
fr: 86%, mr: 43% 	 f1:  1, f2: 27, p1:  7, m1:  0, m2: 52, t1:  0, t2:  2, p2: 11
			 Total:100 					  573 / 3000 completed 

read:122, recent:8 	 mode: recent
fr:  0%, mr: 38% 	 f1:  0, f2:  0, p1: 10, m1:  0, m2: 76, t1:  0, t2:  2, p2: 12
			 Total:100 					  574 / 3000 completed 

read:1545, recent:141 	 mode: recent
fr: 94%, mr: 36% 	 f1:  5, f2: 94, p1:  0, m1:  0, m2:  0, t1:  0, t2:  1, p2:  0
			 Total:100 					  575 / 3000 completed 

read:4, recent:4 	 mode: else
fr:100%, mr:100% 	 f1:  0, f2: 31, p1:  6, m1:  0, m2: 50, t1:  0, t2:  0, p2: 13
			 Total:100 					  576 / 3000 completed 

read:183, recent:22 	 mode: recent
fr: 23%, mr: 55% 	 f1:  1, f2: 88, p1:  1, m1:  0, m2:  0, t1:  0, t2: 10, p2:  0
			 Total:100 					  577 / 3000 completed 

read:1530, recent:22 	 mode: recent
fr: 14%, mr: 36% 	 f1:  0, f2:100, p1:  

fr:  0%, mr: 71% 	 f1:  0, f2:  0, p1: 10, m1:  0, m2: 74, t1:  0, t2: 16, p2:  0
			 Total:100 					  624 / 3000 completed 

read:310, recent:32 	 mode: recent
fr: 47%, mr: 44% 	 f1:  7, f2: 78, p1:  1, m1:  0, m2: 11, t1:  0, t2:  3, p2:  0
			 Total:100 					  625 / 3000 completed 

read:1170, recent:21 	 mode: recent
fr:  5%, mr: 38% 	 f1:  0, f2:  3, p1:  9, m1:  1, m2: 73, t1:  0, t2:  6, p2:  8
			 Total:100 					  626 / 3000 completed 

read:606, recent:25 	 mode: recent
fr: 40%, mr: 24% 	 f1:  1, f2:  6, p1:  9, m1:  0, m2: 30, t1:  0, t2: 54, p2:  0
			 Total:100 					  627 / 3000 completed 

read:3, recent:2 	 mode: else
fr:  0%, mr: 67% 	 f1:  0, f2:  0, p1: 10, m1:  0, m2: 46, t1:  0, t2: 15, p2: 29
			 Total:100 					  628 / 3000 completed 

read:120, recent:34 	 mode: recent
fr: 21%, mr: 50% 	 f1:  2, f2: 56, p1:  4, m1:  0, m2: 34, t1:  0, t2:  4, p2:  0
			 Total:100 					  629 / 3000 completed 

read:2328, recent:303 	 mode: recent
fr: 55%, mr: 30% 	 f1:  0, f2: 68, p

fr:100%, mr: 20% 	 f1:  2, f2: 95, p1:  0, m1:  0, m2:  0, t1:  0, t2:  3, p2:  0
			 Total:100 					  676 / 3000 completed 

read:50, recent:50 	 mode: recent
fr: 20%, mr: 34% 	 f1:  1, f2: 10, p1:  8, m1:  0, m2: 58, t1:  0, t2: 23, p2:  0
			 Total:100 					  677 / 3000 completed 

read:259, recent:59 	 mode: recent
fr: 93%, mr: 51% 	 f1:  0, f2: 31, p1:  6, m1:  0, m2: 58, t1:  0, t2:  2, p2:  3
			 Total:100 					  678 / 3000 completed 

read:16, recent:2 	 mode: else
fr: 62%, mr: 31% 	 f1:  1, f2:  3, p1:  9, m1:  0, m2: 73, t1:  0, t2:  9, p2:  5
			 Total:100 					  679 / 3000 completed 

read:696, recent:263 	 mode: recent
fr: 87%, mr: 14% 	 f1:  1, f2: 44, p1:  5, m1:  0, m2: 26, t1:  0, t2: 24, p2:  0
			 Total:100 					  680 / 3000 completed 

read:2111, recent:315 	 mode: recent
fr: 79%, mr: 35% 	 f1:  3, f2: 74, p1:  2, m1:  0, m2:  2, t1:  0, t2: 19, p2:  0
			 Total:100 					  681 / 3000 completed 

read:111, recent:111 	 mode: recent
fr: 98%, mr: 28% 	 f1:  2, f2: 90, 

fr:  0%, mr: 76% 	 f1:  0, f2:  0, p1: 10, m1:  2, m2: 73, t1:  0, t2: 15, p2:  0
			 Total:100 					  728 / 3000 completed 

read:27, recent:4 	 mode: read
fr:  7%, mr: 41% 	 f1:  1, f2: 60, p1:  3, m1:  0, m2: 30, t1:  0, t2:  6, p2:  0
			 Total:100 					  729 / 3000 completed 

read:18, recent:11 	 mode: recent
fr:  0%, mr: 55% 	 f1:  0, f2:  0, p1: 10, m1:  0, m2: 53, t1:  0, t2: 37, p2:  0
			 Total:100 					  730 / 3000 completed 

read:322, recent:38 	 mode: recent
fr: 39%, mr: 47% 	 f1:  0, f2: 28, p1:  7, m1:  0, m2: 47, t1:  0, t2: 18, p2:  0
			 Total:100 					  731 / 3000 completed 

read:721, recent:104 	 mode: recent
fr:  4%, mr: 46% 	 f1:  1, f2: 10, p1:  8, m1:  0, m2: 50, t1:  0, t2: 31, p2:  0
			 Total:100 					  732 / 3000 completed 

read:32, recent:28 	 mode: recent
fr: 61%, mr: 46% 	 f1:  0, f2: 20, p1:  8, m1:  2, m2: 37, t1:  0, t2:  7, p2: 26
			 Total:100 					  733 / 3000 completed 

read:250, recent:164 	 mode: recent
fr: 26%, mr: 48% 	 f1:  1, f2: 49, p1:

fr:100%, mr: 23% 	 f1:  3, f2: 95, p1:  0, m1:  0, m2:  0, t1:  0, t2:  2, p2:  0
			 Total:100 					  780 / 3000 completed 

read:33, recent:16 	 mode: recent
fr:  0%, mr: 88% 	 f1:  0, f2:  0, p1: 10, m1:  1, m2: 81, t1:  0, t2:  8, p2:  0
			 Total:100 					  781 / 3000 completed 

read:0, recent:0 	 mode: else
fr:  0%, mr:  0% 	 f1:  0, f2:  0, p1: 10, m1:  0, m2:  0, t1:  0, t2:  0, p2: 90
			 Total:100 					  782 / 3000 completed 

read:315, recent:46 	 mode: recent
fr: 91%, mr: 28% 	 f1: 12, f2: 88, p1:  0, m1:  0, m2:  0, t1:  0, t2:  0, p2:  0
			 Total:100 					  783 / 3000 completed 

read:23, recent:1 	 mode: else
fr: 22%, mr: 56% 	 f1:  2, f2: 59, p1:  3, m1:  1, m2: 26, t1:  0, t2:  2, p2:  7
			 Total:100 					  784 / 3000 completed 

read:705, recent:74 	 mode: recent
fr:  4%, mr: 70% 	 f1:  0, f2:100, p1:  0, m1:  0, m2:  0, t1:  0, t2:  0, p2:  0
			 Total:100 					  785 / 3000 completed 

read:424, recent:24 	 mode: recent
fr:100%, mr: 28% 	 f1:  1, f2: 87, p1:  1, m